The following procedures are aimed at exploring and analyzing different databases with the end goal of extracting valuable information and identifying patterns. In particular, the first part focuses on the dataset "vending.csv", provided for the assignment, while the following two parts are centered on datasets obtained from the surveys run on Qualtrics both for the general testing of the proposed hypothesis and for the A/B testing of our final proposals.

# PART A (Analyses of provided dataset)

## Index:    <a class="anchor" id="index"></a>
* [Index](#index)
* [Import of relevant libraries](#zero-bullet)
* [Import of data](#first-bullet)
* [Missing Values and Type Checks](#second-bullet)
* [Correlation Checks](#corr-bullet)
* [Dealing with Excess Columns](#third-bullet)
* [Data Visualization and Outliers Checks](#fourth-bullet)
* [Feature Engineering](#fifth-bullet)
* [Exploratory Data Analysis](#sixth-bullet)
* [First t-test](#seventh-bullet)
* [Cash vs Credit Comparison](#eighth-bullet)
* [Second t-test](#ninth-bullet)
* [Cash-Carbonated vs Credit-Carbonated](#tenth-bullet)
* [Location](#eleventh-bullet)
* [RCoil](#twelfth-bullet)
* [CatBoost](#thirhtenth-bullet)



# Import of relevant libraries <a class="anchor" id="zero-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
%matplotlib inline
np.random.seed(42)

# Import of data <a class="anchor" id="first-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

In [ ]:
db=pd.read_csv('vending.csv')

Brief vizualization the head of the dataset to have a grasp at the data:

In [ ]:
db.head()

In [ ]:
db.shape

# Missing Values and Type Checks <a class="anchor" id="second-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

A check for missing values has then been conducted: 

In [ ]:
db.isnull().sum()

For further analyses, the index of the missing values is retrieved:

In [ ]:
db[db['MPrice'].isnull()].index.tolist()

In [ ]:
db[db['Product'].isnull()].index.tolist()

In [ ]:
mis=db[db['Category'].isnull()].index.tolist()
mis

Seeing that most of the missing values are in the attribute "Category" but not in the attribute "Product", it is possible to retreive the name of the product so that the missing value can be replaced manually leading to no data loss:

In [ ]:
missing=set(db['Product'][mis])
missing

Filling up of NaNs can is done with the following loop:

In [ ]:
for i in mis:
    if db['Product'][i]=='Canada Dry - Ginger Ale':
        db['Category'][i]='Carbonated'
    if db['Product'][i]=='Canada Dry - Ginger Ale & Lemonde':
        db['Category'][i]='Carbonated'
    if db['Product'][i]=='Doritos Dinamita Chile Lemon':
        db['Category'][i]='Food'
    if db['Product'][i]=='Doritos Spicy Nacho':
        db['Category'][i]='Food'
    if db['Product'][i]=='Mini Chips Ahoy - Go Paks':
        db['Category'][i]='Food'
    if db['Product'][i]=='Oreo Mini - Go Paks':
        db['Category'][i]='Food'
    if db['Product'][i]=='Starbucks Doubleshot Energy - Coffee':
        db['Category'][i]='Non Carbonated'
    if db['Product'][i]=='Teddy Grahams - Go Paks':
        db['Category'][i]='Food'

There should still remain 4 missing values in the feature "Category" corresponding to those entries that also have a misisng value in the "Product" feature:

In [ ]:
mis=db[db['Category'].isnull()].index.tolist()
mis

In [ ]:
newdb=db.drop(mis,axis=0)

In [ ]:
newdb.shape

As no further operation is possibile to retrive the 4 missing values left, they have been dropped. Being that they are only 4, out of a total of 6445 observations, the risk of data loss is fairly limited.

Checks of the types of inputs data are then performed:

In [ ]:
newdb.info()

An additional check to see whether the feature "Status" presents a unique value or more than one has then been performed with the result that the column contains only one value (namely "Processed"):

In [ ]:
newdb["Status"].unique()

# Correlation Checks <a class="anchor" id="corr-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>
<br> <br>

In [ ]:
corr_matrix=newdb.corr()

In [ ]:
plt.figure(figsize=(30,15))
sns.heatmap(corr_matrix,annot=True,cmap="YlGnBu")

From the correlation matrix it can be seen that some variables present really high (even perfect in some case) correlation with each other. Correlation can cause issues in several algorithms and models, for this reason, this highly correlated variables will be better explored in the subsequent section. 

# Dealing with Excess Columns <a class="anchor" id="third-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>
<br> <br>

In this subsection, all excess columns will be dropped.
To begin with, it can be seen that Coil, Qty, and Price are present in two differnt variations inside the provided dataset. Given that the correlation between the variations is perfect (Pearson's correlation coefficient=1), as highlighted by the correlation matrix, it is very likely that the variations of the aforementioned columns are identical. An assesment of whether they contain the same information is therefore run:

In [ ]:
sum(newdb["RCoil"]==newdb["MCoil"])

In [ ]:
sum(newdb["RQty"]==newdb["MQty"])

In [ ]:
sum(newdb["RPrice"]==newdb["MPrice"])

The assesment conducted, yielded the result that the information contained in the columns compared are the same. Now the same procedure is also run for "RPrice" and "LineTotal" that, in theory, should present at least some different values:

In [ ]:
sum(newdb["RPrice"]==newdb["LineTotal"])

And indeed they do.

Now, a check to see whether there are some Transaction ID that are repeated in several dates has been conducted:

In [ ]:
df1 = newdb[newdb.groupby('Transaction')['TransDate'].transform('nunique').ne(1)]
df1

Given that the new dataframe is empty, there are no repeated Transaction ID in different days. 

It is now possible to drop the excess columns, meanining those that do not provide additional information to then be used for prediction or data analysis in general:

In [ ]:
n_db=newdb.drop(["Status","Device ID","MCoil","MPrice","MQty","Transaction"],axis=1)

In particular, "MPrice", "MQty", and "MCoil" have been dropped as they provide the same information of "RCoil", "RQty", and "RPrice", while "Status" has been dropped because it presents a single values, as showed in the section above. "Devide ID" was dropped as the name of the vending machine is present in the dataset, and so having a feature containing the ID would just be a duplicate information. Last but not least, "Transaction" was dropped aswell since it is just a number indexing transactions, it does not provide additional useful information. 

# Data Visualization and Outliers Checks <a class="anchor" id="fourth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>
<br> <br>

Histograms of remaining data have been plotted to see their approximate distributions:

In [ ]:
n_db.hist(bins=50, figsize=(30,30))

Data seem in line with the expectationa and do not seem to be very unbalanced. Given that no detailed information about the nature of the data are available, no further action has been taken to rescale/normalize the data for the time being. 

A check for the presence of outliers has then been performed. The check has been, to begin with, done graphically with boxplots. Progressively, columns have been removed so that also visualisation of features with smaller magnitude was possible:

In [ ]:
newdb_no_trans=newdb.copy()
newdb_no_trans=newdb_no_trans.drop("Transaction",axis=1)
plt.figure(figsize=(30, 20))
sns.boxplot(data=newdb_no_trans)

In [ ]:
newdb_no_big=newdb.copy()
newdb_no_big=newdb_no_big.drop(["Transaction","RCoil","MCoil"],axis=1)
plt.figure(figsize=(30, 20))
sns.boxplot(data=newdb_no_big)

In [ ]:
newdb_no_big2=newdb.copy()
newdb_no_big2=newdb_no_big2.drop(["Transaction","RCoil","MCoil","RQty","MQty"],axis=1)
plt.figure(figsize=(30, 20))
sns.boxplot(data=newdb_no_big2)

There seems not to be any absurd value in any of the feature. To double check this, the feature "TransTotal", which will also be the target variable of the analysis, is analysed more in depth:

In [ ]:
n_db[n_db["TransTotal"]>6]

It is possible to notice that the values greter than 6 are still plausible values, as a consequence, no row has been dropped to prevent information loss. 

# Feature Engineering <a class="anchor" id="fifth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

Some feature engineering was then perform to prepare the data to be fed to algotithms and models for analysis.

To begin with, the date feature was split:

In [ ]:
n_db["Date_Split"]=n_db["TransDate"].str.split(", ")

In [ ]:
n_db[["Day_of_Week","Month+Day","Year"]]=pd.DataFrame(n_db.Date_Split.tolist(),index=n_db.index)

At this point, some columns contain same information about the data as others. A drop is therefore necessary:

In [ ]:
n_db=n_db.drop(["TransDate","Date_Split"],axis=1)

Days were then split between Festive and Working days. Days were classified as festive if they were either a saturday, a sunday, or a New Jersey State holiday (https://www.state.nj.us/nj/about/facts/holidays/). Holidays were considered up to July, month of the latest observation in the dataset. The code to perform such operation is as follows:

In [ ]:
holiday=["January 1","January 17","February 21","April 15","May 30","June 17","July 4"]
n=(n_db["Day_of_Week"]=="Saturday") | (n_db["Day_of_Week"]=="Sunday") 
m=n_db["Month+Day"].isin(holiday)

In [ ]:
n_db["Festive"]=np.where(n|m,1,0)

The column containing the year was also dropped as it contained do additional information given that all observation in the dataset are from 2022:

In [ ]:
n_db=n_db.drop(["Year"],axis=1)

Once the days were classified as festive or nor, a column containing the month was created:

In [ ]:
l=[]
for i in range(len(n_db["Month+Day"])):
    l.append(n_db["Month+Day"].iloc[i].split(" ")[0])
n_db["Month"]=l

And, once again, columns containing duplicate information were dropped:

In [ ]:
n_db = n_db.drop(["Month+Day","Prcd Date"],axis=1)

The precise day date of the month was dropped as it was deemed not that relevant once the indication of the month, of the week day, and of whether that day was festive or not was included. 

## Exploratory Data Analysis <a class="anchor" id="sixth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

In this section, some exploratory data analysis has been conducted in order to deepen the knowledge of the data at hand and to try to identify patterns or charcateristics linking different purchases.

The "TransTotal" has been analysed as it is the variable of interest of the analysis. The means has been plotted according to different weekdays:

In [ ]:
n_db.groupby('Day_of_Week')["TransTotal"].mean().plot()

There seems to be a difference in means between different days. To see whether this difference is really due to "Day_of_Week" or rather to other factors, the "Type" of the transaction was also included: 

In [ ]:
n_db.groupby(['Type', 'Day_of_Week'])["TransTotal"].mean()

There seems to be a relevant difference between the average transaction total for purchases made with card and the ones made with cash. In particular:

In [ ]:
n_db.groupby('Type')["TransTotal"].mean()

# First t-test <a class="anchor" id="seventh-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

To see whether the empirically discovered difference is statistically significant, a t-test has been run:

In [ ]:
from scipy import stats
stats.ttest_ind(a=n_db[n_db['Type'] == 'Cash'].TransTotal, b=n_db[n_db['Type']== 'Credit'].TransTotal, equal_var=False)

Given that the p-value is approximately 0, the null hypothesis of non-significance of the difference has been rejected and so there is a statistically significant difference between the transactions done with card and with cash. 

At this point, since there is a statistically significant difference, a deeper comparison has been run on Cash VS Credit in the following section.

# Cash vs Credit Comparison <a class="anchor" id="eighth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

In this section, a deeper analysis has been run to understand whether other features can be the main cause in the differnce between average transaction total when paying with card with respect to when paying with cash.

After this point there isn't any operation on the original ready n_db dataset, just some exploratory analysis. 

First of all, a comparison within categories has been carried out:

In [ ]:
d_cash = n_db[n_db['Type'] == 'Cash']
d_credit = n_db[n_db['Type'] == 'Credit']

In [ ]:
d_cash['TransTotal'].hist(), d_credit['TransTotal'].hist()

In [ ]:
d_credit.groupby('Category')["TransTotal"].mean()

In [ ]:
d_credit.groupby('Day_of_Week')["TransTotal"].mean()

In [ ]:
d_cash.groupby('Day_of_Week')["TransTotal"].mean()

In [ ]:
d_cash.groupby('Category')["TransTotal"].mean(), d_credit.groupby('Category')["TransTotal"].mean()

There seems to be a difference between the average transtotal with cash and with credit in all categories of products, let us start by seeing whether the one in the "Carbonated" category is indeed statistically significant.

# Second T-test <a class="anchor" id="ninth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

In [ ]:
from scipy import stats
stats.ttest_ind(a=d_cash[d_cash['Category'] == 'Carbonated'].TransTotal, b=d_credit[d_credit['Category'] == 'Carbonated'].TransTotal, equal_var=False)

The t-test yield a p-value of 3.392838283865919e-38, meaning that the null hypothesis of non-significance is rejected. Given that there is a significant difference between the average transaction total with cash and credit card when it comes to "Carbonated" drinks, a further analysis is conducted to deepen the understanding.

# Cash-Carbonated vs Credit-Carbonated <a class="anchor" id="tenth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

Here follows some comparisons between the carbonated drinks that are bought with cash and those that are bought with credit card.

In [ ]:
carbonated_cash = d_cash[d_cash['Category'] == 'Carbonated']
carbonated_credit =d_credit[d_credit['Category'] == 'Carbonated']

In [ ]:
l = carbonated_cash.groupby('Product')['RPrice'].mean()-carbonated_credit.groupby('Product')['RPrice'].mean() == 0
#carbonated["Product"][l]
l = l[l==False]
la = l[l==False].index

In [ ]:
carbonated_differ = n_db.loc[n_db['Product'].isin(la)]
carbonated_differ['Product'].value_counts()

In [ ]:
carbonated_differ_g = carbonated_differ.drop(['Month','TransTotal','Festive','Location','Type','Machine','Category','RQty','LineTotal','Day_of_Week'],axis=1)
carbonated_differ_g

In [ ]:
carbonated_differ_g.groupby('Product')['RPrice'].hist()

In [ ]:
carbonated_differ_g.groupby('Product')['RPrice'].value_counts()

In [ ]:
cola_zero = carbonated_differ.loc[carbonated_differ['Product'] == 'Coca Cola - Zero Sugar']
cola_zero.groupby('RPrice')['Location'].value_counts()

In [ ]:
cola_zero.groupby('RPrice')['Month'].value_counts()

In [ ]:
carbonated = n_db[n_db['Category']=='Carbonated']
carbonated.groupby('Product')['RPrice'].mean()

In [ ]:
carbonated = n_db[n_db['Category']=='Carbonated']
carbonated.groupby('RPrice')['Product'].count()

In [ ]:
carbonated.groupby('Product')['RPrice'].sum()

From the analyses carried out above, what was discovered is that different products have different prices for unknown reasons. Therefore, it is not possible to conclude whether carbonated drinks are those really driving the difference between cash and credit or not. 

The same holds true also for the other categories of products, for easiness of presentation codes are not reported in the final version but the results are in line with what happened with carbonated drinks.

## Location  <a class="anchor" id="eleventh-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

At this point, an analysis of how the location might have had an impact of the transaction total has been carried out.

In [ ]:
n_db.Location.value_counts()

In [ ]:
n_db.groupby('Location')['TransTotal'].mean()

Here the variation seems to be way smaller that the one obtained for the cash vs credit comparison performed above.

In [ ]:
n_db.groupby(['Location','Day_of_Week'])['TransTotal'].sum()

From the analysis above, it is possible to confirm what was already foreseeable with logic: transaction totals are higher in the mall during weekends and in the offices/library during weekdays. This was imaginable since the library and the offices are working venues while the mall is mainly a leisure one.

## RCoil  <a class="anchor" id="twelfth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>

Here we tried to analyse whether the position of the product in the vending machine has some relevance.

In [ ]:
n_db.groupby('RCoil')['TransTotal'].sum().plot()

From this representation it seems that the most used coils are the ones approximately at the middle. Let us try to gain some more information by grouping the coils and plotting an heatmap. 

A variable summarizing the coils has been created. This variable, in particular, contains the number of the row of the coil used (here we assumed that the vending machines at hand have a standard configuration: 10 coils per row):

In [ ]:
n_db_c=n_db.copy()

In [ ]:
n_db_c['hor_eye'] = np.nan

In [ ]:
for i in range(len(n_db_c)):
    n_db_c['hor_eye'].loc[n_db_c.index[i]] = int(str(n_db_c['RCoil'].loc[n_db_c.index[i]])[1])

This new variable is than use to plot an heatmap to use the usage of coils:

In [ ]:
a = n_db_c.groupby('RCoil')['TransTotal'].sum()
b = a.to_frame()
data = pd.DataFrame({'RCoil': range(110,170),
                   'Total': 0},
                   columns = ['RCoil', 'Total'])
data = pd.merge(b, data, left_on="RCoil", right_on="RCoil")
data.drop(['Total'], axis=1, inplace= True)
data['Row'] = np.nan
data['Column'] = np.nan
for i in range(len(data)):
    data['Row'].loc[data.index[i]] = int(str(data['RCoil'].loc[data.index[i]])[1])
    data['Column'].loc[data.index[i]] = int(str(data['RCoil'].loc[data.index[i]])[2])
data.drop(['RCoil'], axis=1, inplace = True)
datas = pd.DataFrame(np.nan, index=[i for i in range(1,7)], columns=[i for i in range(0,10)])
for i in range(len(data)):
    a = data['Row'].loc[data.index[i]]
    b = data['Column'].loc[data.index[i]]
    c = data['TransTotal'].loc[data.index[i]]
    datas[a][b] = c
datas.drop([0,6,7,8,9], axis =1, inplace = True)
datas

In [ ]:
ax = sns.heatmap(datas, cmap = 'YlOrRd')
ax.set(xlabel="Columns", ylabel="Rows")
ax.xaxis.tick_top()

This heatmap seems to verify what was also seen before. Though, given that no extra information is provided about the configurations of the machines, we decided not to further investigate in this respect as it might be the case that the central coils are the most used because some machines might have only them, for instance. Still, these results as well as the ones obtained for the previous features, can be useful to develop new strategies and theories if combined with further information.  

## CatBoost  <a class="anchor" id="thirteenth-bullet"></a>
[Index](#index) <a class="anchor" id="index"></a>



To verify what has been previously discovered through data exploration procedures, a CatBoost has been run. CatBoost is a machine learning algorithm that uses gradient boosting on decision trees. This procedure was selected as clustering algorithms did not perform well given the huge number of categorical varibales present in the dataset. For this reason, a tree based approach was selected in order to identify what are the features generating the greatest difference between observations and so granting an higher information gain once split on those variables has been performed. 

In [ ]:
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

To begin with, the target feature is isolated from the rest of the dataset and the train/test split is performed:

In [ ]:
target = n_db["TransTotal"]

In [ ]:
n_db=n_db.drop(["TransTotal"],axis=1)

In [ ]:
n_db=n_db.drop(["LineTotal", "RPrice"],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(n_db, target, test_size = 0.2, random_state=5)

In [ ]:
train_dataset = cb.Pool(X_train, y_train, cat_features = ["Location", "Machine", "Product", "Category","Type", "Day_of_Week", "Festive", "Month", "RCoil"])
test_dataset = cb.Pool(X_test, y_test, cat_features = ["Location", "Machine", "Product", "Category","Type", "Day_of_Week", "Festive", "Month", "RCoil"])

In [ ]:
model = cb.CatBoostRegressor(loss_function="RMSE")

A grid-search has then been run to identify the optimal parameters among the ones fed to the algorithm:

In [ ]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.025,0.05,0.075, 0.1],
        'depth': [2, 4, 6, 8, 10],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid,train_dataset)

The moel has the been fit on the optimal hyperparameters:

In [ ]:
model = cb.CatBoostRegressor(depth= 10,l2_leaf_reg= 1,iterations=200,learning_rate= 0.075,loss_function="RMSE")
model.fit(train_dataset)

And the feature importances have been obtained:

In [ ]:
sorted_feature_importance = model.feature_importances_.argsort()
plt.barh(n_db.columns[sorted_feature_importance], model.feature_importances_[sorted_feature_importance], 
        color='turquoise')
plt.xlabel("CatBoost Feature Importance")

Once the importances have been devised, let us asses the performance of the algorithm:

In [ ]:
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

# PART B (Preparation of the datasets to be used on STATA to analyze survey results)

The starting point is the import of the survey dataset:

In [ ]:
df = pd.read_csv("H-FARM INNOVATION business case_November 19, 2022_11.57.csv", sep = ";")

Where the column concerning the card is null, we associated a 1 in the newly created "Non_user" column:

In [ ]:
l = np.where(df["Q10"].isnull())[0]

In [ ]:
df["Non_user"] = 0

In [ ]:
for i in l:
    df.at[i, "Non_user"] = 1

A similar procedure was carried out for the credit card users:

In [ ]:
df["Card_y"] = 0

In [ ]:
n = np.where(df["Q10"]==1.0)

In [ ]:
for i in n:
    df.at[i, "Card_y"] = 1

and also for the cash users:

In [ ]:
df["Card_n"] = 0

In [ ]:
m = np.where(df["Q10"]==0.0)

In [ ]:
for i in m:
    df.at[i, "Card_n"] = 1

Then, the column concerning card ussage has been dropped since now its information is stored in the previously created columns:

In [ ]:
df = df.drop("Q10", axis=1)

We created the df1 dataset containing only student observations and the df2 containing only worker observations:

In [ ]:
df1 = df.drop("Usage Workplace", axis=1)

In [ ]:
df1 = df1.dropna()

In [ ]:
df2 = df.drop("Usage Library", axis=1)

In [ ]:
df2 = df2.dropna()

In [ ]:
df1.to_csv("df_stud.csv")
df2.to_csv("df_work.csv")

# PART C (Preparation of the datasets to be used on STATA to analyze A/B survey's results)

We import the relevent datasets and remove excess rows:

In [ ]:
df1 = pd.read_csv("A_B test_November 26, 2022_07.00.csv", sep = ";")

In [ ]:
df1 = df1.drop([0,1,2], axis = 0)

In [ ]:
df2 = pd.read_csv("A_B test 2_November 26, 2022_07.02.csv", sep = ";")

In [ ]:
df2 = df2.drop([0,1,2], axis = 0)

We concatenate the two datasets to build a new one:

In [ ]:
df_final = df1.append(df2)

And we create a new column containing the offer type showed to the respondent:

In [ ]:
l = [0]*100+[1]*103

In [ ]:
df_final["offer_type"] = l

We then renamed the columns for easiness of procedures:

In [ ]:
df_final.rename(columns={"Q3":"age", "Q4":"NJ resident", "Q5":"occupation", "Q7_1": "vending_user", "Q9": "nonuser_to_app", "Q10_1":"card_yn", "Unnamed: 6":"cash_to_app"}, inplace=True)

and dropped rows corresponding to those who pay with credit card:

In [ ]:
df_final = df_final.dropna(axis=0, subset=["nonuser_to_app", "cash_to_app"], how="all")

Before exporting the csv to be used on STATA, we combined the observations for non users with the ones for cash users in a single column named "propensity":

In [ ]:
df_final["propensity"] = df_final["nonuser_to_app"].fillna(df_final["cash_to_app"])

In [ ]:
df_final.to_csv("df_final_ab.csv")